<a href="https://colab.research.google.com/github/neuhausmatheus/vicuna/blob/main/Vicuna_13B_PRO_GPU_RAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**You may encounter an error when installing flash-attn. I couldn't figure it out. Maybe you can.**

Testing GPU access on Colab

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Apr  7 15:11:06 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P0    48W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Testing Memory access on Colab

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 89.6 gigabytes of available RAM

You are using a high-RAM runtime!


Preparation...

In [ ]:
%pip install torch==1.13.1+cu116 torchvision==0.14.1+cu116 torchaudio==0.13.1 --extra-index-url https://download.pytorch.org/whl/cu116
!cd ~
!git clone https://github.com/huggingface/transformers.git && cd transformers && git checkout cae78c46 && pip install .
# Install fastchat
!pip3 install --upgrade pip
!git clone https://github.com/lm-sys/FastChat && cd FastChat && pip install -e .
%pip install einops
!mkdir checkpoints
!wget https://huggingface.co/datasets/anon8231489123/ShareGPT_Vicuna_unfiltered/resolve/main/flash_attn-0.2.8-cp39-cp39-linux_x86_64.whl
%pip install flash_attn-0.2.8-cp39-cp39-linux_x86_64.whl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu116
fatal: destination path 'transformers' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
fatal: destination path 'FastChat' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
mkdir: cannot create directory ‘checkpoints’: File exists
--2023-04-07 15:18:28--  https://huggingface.co/datasets/anon8231489123/ShareGPT_Vicuna_unfiltered/resolve/main/flash_attn-0.2.8-cp39-cp39-linux_x86_64.whl
Resolving huggingface.co (huggingface.co)... 52.84.125.99, 52.84.125.117, 52.84.125.98, ...
Connecting to huggingface.co (huggingface.co)|52.84.125.99|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/58/74/5874e8234c

In [ ]:
!wget https://raw.githubusercontent.com/oobabooga/text-generation-webui/main/download-model.py
!mkdir models
!wget https://huggingface.co/datasets/anon8231489123/ShareGPT_Vicuna_unfiltered/resolve/main/ShareGPT_V2_unfiltered_cleaned_split.json
!python download-model.py decapoda-research/llama-13b-hf

--2023-04-07 15:18:47--  https://raw.githubusercontent.com/oobabooga/text-generation-webui/main/download-model.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9424 (9.2K) [text/plain]
Saving to: ‘download-model.py’

download-model.py   100%[===================>]   9.20K  --.-KB/s    in 0s      

2023-04-07 15:18:47 (102 MB/s) - ‘download-model.py’ saved [9424/9424]

--2023-04-07 15:18:47--  https://huggingface.co/datasets/anon8231489123/ShareGPT_Vicuna_unfiltered/resolve/main/ShareGPT_V2_unfiltered_cleaned_split.json
Resolving huggingface.co (huggingface.co)... 52.84.125.99, 52.84.125.117, 52.84.125.98, ...
Connecting to huggingface.co (huggingface.co)|52.84.125.99|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-

**Manually edit tokenizer_config.json to: {"bos_token": "", "eos_token": "", "model_max_length": 2048, "tokenizer_class": "LlamaTokenizer", "unk_token": ""}**

**Enter wandb api key**

In [ ]:
%pip install wandb
import wandb
wandb.login()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

**8 x A100 80gb training run** 

In [ ]:
!export CUDA_LAUNCH_BLOCKING=1 && echo $CUDA_LAUNCH_BLOCKING && torchrun --nnodes=1 --nproc_per_node=8 --master_port=21001 \
    FastChat/fastchat/train/train.py \
    --model_name_or_path models/decapoda-research_llama-13b-hf \
    --data_path ShareGPT_unfiltered_cleaned_split.json \
    --bf16 True \
    --output_dir ./checkpoints \
    --num_train_epochs 3 \
    --per_device_train_batch_size 4 \
    --per_device_eval_batch_size 4 \
    --gradient_accumulation_steps 1 \
    --evaluation_strategy "no" \
    --save_strategy "steps" \
    --save_steps 1200 \
    --save_total_limit 100 \
    --learning_rate 2e-5 \
    --weight_decay 0. \
    --warmup_ratio 0.03 \
    --lr_scheduler_type "cosine" \
    --logging_steps 1 \
    --fsdp "full_shard auto_wrap" \
    --fsdp_transformer_layer_cls_to_wrap 'LlamaDecoderLayer' \
    --tf32 True \
    --model_max_length 2048 \
    --gradient_checkpointing True \
    --lazy_preprocess True

1
*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
2023-04-07 15:29:09.951532: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-04-07 15:29:09.952351: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-04-07 15:29:10.047862: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-04-07 15:29:10.240560: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-04-07 15:29:11.188418: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-04-07 15:29:11.415681: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] 